# Analytics Best Practices - Staying Light on Transformations

This notebook illustrates a very simple yet powerful idea I've been exploring over the last couple of years.

**The Key Idea**: Central data teams almost always struggle with the challenge of having

- dozens of "source domains" inside their data structures, say "orders", "customers", ... all of which need a decent amount of knowledge
to properly transform them into something useful
- dozens of "target domains" like marketing, sales,... which again need special knowledge to transform source data into information
fitting their needs.

So at the center, the problem is that a **low number of central data teams (low compared to the number of source & target teams) needs
to understand a multiple of other domains to get their work done**. In my opinion there are two options to solve this problem (once it 
becomes a true problem). 
1. Keep the domain stuff inside the domains => that's called "distributed data domain ownership" and is the first step towards a data mesh.
2. <= focus of this piece: Pass the domains through as much as possible, be "light on transformations", if not completely possible, 
keep them as close to the endusers as possible.



*Note: If many domains aren't a problem for you, then you probably don't need this! It's entirely fine to make stuff work. Usually that means
you have well organized domains, or a low number of domains, or low complexity inside the domains. You might also chose to have 
"one domain expert" for the major domains, or you might chose to decentralize the analysis & possibly the transformation part of the work.
All of which are organizational solutions to the problem.
Here, I'm talking about a solution that works out-of-the-box for the existing centralized data team.*

Enough talk, let's get started...

## Let's do some dependency magic

In [35]:
# Let's get our dependencies
import sys
!{sys.executable} -m pip install pandas
import pandas as pd
from datetime import datetime

# some constants
DATA_FILE_PATH="work/data.csv"

## Let's get started! With an Example!

In [41]:
df = pd.read_csv(DATA_FILE_PATH)
print("This is our dataframe...\n", df.head(5))

This is our dataframe...
   CRMRep  day  Order Status
0  James    1      1      A
1  James    1      2      F
2    Eve    2      3      A
3    Bob    3      4      B
4    Eve    4      5      A


This is our input data, the simplest possible I could find.

**Our task** is to produce a dashboard for now just with one table. Our stakeholder would like to see...
- The orders for each Sales Rep, for each of the status (A=ordered, B= shipped, F=returned)

In [42]:
## 1. First let's do some prettifying on the days, We're in 2022 in the month 1, so we can actually
## turn this into a proper date
df['day'] = df['day'].apply(lambda x: datetime(2022,1,x)) 


## 2. That looks already much nicer... Next, let's tackle the order status, let's map that to its speaking name
df['Status'] = df['Status'].apply(lambda x: "ordered" if x=="A" else ("shipped" if x=="B" else "returned"))


## 3. The wording "CRMRep" isn't so nice, let's change that to the one used by the stakeholder "Sales Rep"
df=df.rename(columns={"CRMRep": "SalesRep"})


## 4. finally, we're going to do the groupying 
print(df.groupby(by=["SalesRep", "Status"]).count().drop(columns="day"))




                   Order
SalesRep Status         
Bob      shipped       1
Eve      ordered       2
James    ordered       1
         returned      1


### How This Could Be Improved by Staying Light on Transformation ###

I'm going to teach you the idea of "stay light on data transformations" using the example above.

**Problem 1**: In (1) we've done some really naive transformation, mapping the "day" into the year 2022, month 01 to "prettify" it. But as you might've already realized, this might break, once we reach another month. Of course the question is: How does the source data 
look like in Feb 2022? And the answer is: We probably don't know. Soooo... the solution?

**The Solution?**: Try to avoid transformations, period. 

Let's remove this one, and see the obvious caveats to this one...

In [44]:
df = pd.read_csv(DATA_FILE_PATH)

## 1. We simply are not going to do the "prettifying". If the source tells us it's "day 1" we display it as day 1.

## 2. That looks already much nicer... Next, let's tackle the order status, let's map that to its speaking name
df['Status'] = df['Status'].apply(lambda x: "ordered" if x=="A" else ("shipped" if x=="B" else "returned"))


## 3. The wording "CRMRep" isn't so nice, let's change that to the one used by the stakeholder "Sales Rep"
df=df.rename(columns={"CRMRep": "SalesRep"})


## 4. finally, we're going to do the groupying 
print(df.groupby(by=["SalesRep", "Status"]).count().drop(columns="day"))

                   Order
SalesRep Status         
Bob      shipped       1
Eve      ordered       2
James    ordered       1
         returned      1


**The benefits?**: First of all, stuff doesn't break. But in this case, we also removed complexity from our transformation,
roughly 25% which is a lot in my opinion.

Let's tackle the next problem..

**Problem 2**: In 2, we did another transformation, we mapped numbers to speaking names. This seems to be a necessary transformation or does it? The problem is, that our central data team writing these transformations usually isn't 100% up to speed with the semantics of the source domains. How could they, there are dozens of source domains for every data team in the typical company.

So it will happen, that stuff changes. OR that the data team misses one order status (what about C,D,E?). Or that the source team decides to change "A" to "reserved/put into basket". 

. Soooo... the solution?

**The Solution?**: Try to avoid transformation, as in (1). OR we could do something else, we could do the transformation, but do it "close to the end-user". In this case, we could simply display the transformation we chose to do such that a possible user can pick up on wrongly mapped statuses. 

(Alternative 2b: If we'd chosen to avoid this transformation, we would've followed the simple idea of "keeping the semantics inside their domains")

Let's do that.

In [59]:
df = pd.read_csv(DATA_FILE_PATH)

## 1. We simply are not going to do the "prettifying". If the source tells us it's "day 1" we display it as day 1.

## 2. That looks already much nicer... Next, let's tackle the order status, let's map that to its speaking name,
## but this time we're going to make the transformation "close to the end-user"

mapping = {'Status': ["A", "B","F"], 'Status_speaking_name': ["ordered", "shipped","returned"]}
mapping_df = pd.DataFrame(data=mapping)
#print(df)
#print(mapping_df)

joined_df = df.join(mapping_df.set_index("Status"), on="Status")

## 3. The wording "CRMRep" isn't so nice, let's change that to the one used by the stakeholder "Sales Rep"
joined_df=joined_df.rename(columns={"CRMRep": "SalesRep"})



## 4. finally, we're going to do the groupying 
print(joined_df.groupby(by=["SalesRep", "Status_speaking_name"]).count().drop(columns=["day","Status"]))
print("We're using the following mapping table:", mapping_df)

                               Order
SalesRep Status_speaking_name       
Bob      shipped                   1
Eve      ordered                   2
James    ordered                   1
         returned                  1
We're using the following mapping table:   Status Status_speaking_name
0      A              ordered
1      B              shipped
2      F             returned


**The benefits?**: Since we chose to still to a transformation, stuff might still break.
But we did encapsulate the mapping logic into one table which reduces the complexity by decoupling this part. 
If we now get an order status "C,D,E" nothing breaks. In the first implementation it would break. Instead we would get a "NULL" which would 
signal exactly the right thing: The mapping is not complete.

Second, we're making our transformation transparent which means every end-user can easily see what's going on and possibly
correct us.

**Problem 3**: You noticed we renamed "CRMRep" to "SalesRep" because it was the language our stakeholder was using, and because it 
sounds so much better. And it does.

Except, it's entirely possible (and has happened to me in the past) that suddenly marketing also starts to use the CRM System and suddenly
there also is a "MarketingRep". Now "SalesRep" and "MarketingRep" are simply attributes of the CRMRep, and the CRMRep is still the CRMRep.

**The key lesson here is:** There is always a reason a source team names things the way they do. Even if they don't have one,
they will have one in the future. 

**The Solution**: The solution is another idea which is to "stay close to the source". Meaning we try to keep everything as it is on
the source level, not just what we "perceive as semantics" (see problem 2). However we still might want to provide our endusers with a
proper naming... So for this example we can have it both by providing transparency (see the pattern?).

In [73]:
df = pd.read_csv(DATA_FILE_PATH)

## 1. We simply are not going to do the "prettifying". If the source tells us it's "day 1" we display it as day 1.

## 2. That looks already much nicer... Next, let's tackle the order status, let's map that to its speaking name,
## but this time we're going to make the transformation "close to the end-user"

mapping = {'Status': ["A", "B","F"], 'Status_speaking_name': ["ordered", "shipped","returned"]}
mapping_df = pd.DataFrame(data=mapping)

joined_df = df.join(mapping_df.set_index("Status"), on="Status")

## 3. The wording "CRMRep" isn't so nice, let's change that to the one used by the stakeholder "Sales Rep", This is the renaming 
## "module"
joined_df=joined_df.rename(columns={"CRMRep": "SalesRep"})
joined_df['source_name_SalesRep'] = "CRMrep"


## 4. finally, we're going to do the groupying 
agg_df = joined_df.groupby(by=["SalesRep", "Status_speaking_name", "source_name_SalesRep"]).count().drop(columns=["day","Status"])


print(agg_df)
print("We're using the following mapping table:", mapping_df)

                                                    Order
SalesRep Status_speaking_name source_name_SalesRep       
Bob      shipped              CRMrep                    1
Eve      ordered              CRMrep                    2
James    ordered              CRMrep                    1
         returned             CRMrep                    1
We're using the following mapping table:   Status Status_speaking_name
0      A              ordered
1      B              shipped
2      F             returned


Wow! That looks different right? I hope the final example also shows you the three drivers of benefits.

**Benefit drivers**: The benefits are driven through 

- transparency for end-users 
- transparency for data developers
- reduction of complexity by decoupling for end-users
- reduction of complexity by decouplting for developers

**Benefits explained for Solution 3**:

By making the source name transparent, endusers know what this column is referring to in the CRM system. If the change in the CRM system is 
proposed, they might even raise the issue of "breaking reports & dashboards". Or they might not. But they will surely know how to fix it.

By making this transparent to the developers, even someone who never touched this pipeline can see, with a look onto the dashboard,
what might be broken if suddenly "Marketing Reps" appear in the SalesRep column.

Finally, by decoupling this logic into a "3" block, we got a nicely wrapped decoupled part for renaming. It's now easy to stuff
all renaming into this block.